In [1]:
import requests
from lxml import html
from copy import deepcopy
import re
import numpy as np
import pandas as pd

## TODO:
## 1. 下一页
## 2. 建造时间
## 3. 价格时间 log
## 4. 可视化

In [2]:
def check_null(x):
    if len(x) == 0:
        return [""]
    else:
        return x

In [3]:
def trim_to_digit(x):
    # input: a list
    temp = []
    for t in x:
        temp.append("".join(c for c in t if c.isdigit()))
    return temp

In [4]:
base_url = 'http://sh.lianjia.com/ershoufang/rs'
iters = [
    # 汇师小学
    '东方曼哈顿',
    '泰德苑',
    '青之杰花园',
    '文定天下苑',
    '元福大厦',
    '明华苑',
    '实业公寓',
    '宏汇大厦',
    '裕德大夏',
    '中漕小区',
    '徐虹路112弄',
    '裕德路66弄',
    '中原新村',
    '气象局小区',
    '文定路87号',
    
    # 高安路一小
    '集雅公寓',
    '景福苑',
    '首长公寓',
    '欧江大厦',
    '吴兴公寓',
    '吴兴大楼',
    '天平小区',
    '衡山路58弄',
    '吴兴路21弄',
    '华绮公寓', 
    '公安大楼徐汇',
    '高安路9弄',
    '衡山路696弄',
    '华龙公寓徐汇',
    '久安小区',
    '百安小区',
    
    # 向阳小学
    '嘉御庭',
    '梵高馆',
    '嘉善公寓',
    '年平花苑',
    '梧桐花园',
    '金环大厦',
    '阳光四季公寓',
    '裕华大厦',
    '鼎新大楼公寓',
    '锦岩小区',
    '世家小区',
    '建国西路211弄',
    '建西小区', 
    '襄阳南路388弄',
    '襄阳南路452弄',
    '建国西路233弄',
    
    # 建襄小学
    '芝大厦',
    '高欣公寓',
    '岳阳路79弄',
    '伊泰利大厦',
    '永嘉路485弄',
    '联业大夏',
    '岳阳路77弄',
    '太原路76号',
    '岳阳大楼',
    '岳阳路200弄',
    '永嘉大楼',
    '永康路141弄'
]



total_prices = []
idxs = []
prices_per_unit = []
xiaoqus = []


In [5]:
def single_parse(tree, xiaoqu, idxs, xiaoqus, total_prices, prices_per_unit):

    items = tree.xpath("//div[@class='info']")
    for i in range(len(items)):
        leaf = deepcopy(items[i])
        idx = str(leaf.xpath("//div[@class='prop-title']/a/@key")[0])
        total_price = int(leaf.xpath("//span[@class='total-price strong-num']/text()")[0])
        price_per_unit = leaf.xpath("//span[@class='info-col price-item minor']/text()")
        price_per_unit = int(trim_to_digit(price_per_unit)[0])

#         print(idx, total_price, price_per_unit)
        total_prices.append(total_price)
        idxs.append(idx)
        prices_per_unit.append(price_per_unit)
        xiaoqus.append(xiaoqu)
    return (idxs, xiaoqus, total_prices, prices_per_unit)
    
    


In [6]:
for i in iters:
    url = base_url + i
    r = requests.get(url)
    tree = html.fromstring(r.text)
    (idxs, xiaoqus, total_prices, prices_per_unit) = single_parse(tree, i, idxs, xiaoqus, total_prices, prices_per_unit)

In [7]:
results = pd.DataFrame({'idxs': idxs, 
                        'xiaoqus': xiaoqus,
                       'total_prices': total_prices,
                       'prices_per_unit': prices_per_unit})

In [8]:
results.dtypes


idxs               object
prices_per_unit     int64
total_prices        int64
xiaoqus            object
dtype: object

In [11]:
results.loc[(results.total_prices <= 500), :]

,idxs,prices_per_unit,total_prices,xiaoqus
59,sh4553753,158238,460,裕德路66弄
60,sh4307884,130987,495,裕德路66弄
65,sh4737318,125822,440,中原新村
66,sh4745075,165837,500,中原新村
67,sh4721513,158050,480,中原新村
68,sh4343328,164636,500,中原新村
83,sh4754171,130140,500,天平小区
87,sh4788356,136986,500,天平小区
99,sh4677358,118557,490,公安大楼徐汇
113,sh4605344,162962,440,百安小区
